## __Check first before starting__

In [1]:
import os

# Change the working directory to the project root
# Working_directory = os.path.normpath("C:/Users/james/OneDrive/文件/Continual_Learning")
Working_directory = os.path.normpath("/mnt/mydisk/Continual_Learning_JL/Continual_Learning/")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: /mnt/mydisk/Continual_Learning_JL/Continual_Learning


## __All imports__

In [2]:
# Operating system and file management
import os
import shutil
import contextlib
import traceback
import gc
import glob, copy
from collections import defaultdict
import subprocess
import time
import re, pickle

# Jupyter notebook widgets and display
import ipywidgets as widgets
from IPython.display import display

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Plotting and visualization
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler

# Machine learning and preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume

# Mathematical and scientific computing
import math
from scipy.ndimage import gaussian_filter1d

# Type hinting
from typing import Callable, Tuple

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

## __Dataset Setup & Preprocessing__

### 📥 Step 1 — Set dataset path and load features & labels
We will load the 561-dimensional features and original activity labels for both training and testing.

In [3]:
# Set working directory manually before running this cell if needed
BASE_DIR = "Class_Incremental_CL/HAR_CIL/har_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 活動對應（原始標籤）
activity_label_map = {
    1: "WALKING",
    2: "WALKING_UPSTAIRS",
    3: "WALKING_DOWNSTAIRS",
    4: "SITTING",
    5: "STANDING",
    6: "LAYING"
}

# Load feature and label data
X_train = np.loadtxt(os.path.join(TRAIN_DIR, "X_train.txt"))
y_train = np.loadtxt(os.path.join(TRAIN_DIR, "y_train.txt")).astype(int)
X_test = np.loadtxt(os.path.join(TEST_DIR, "X_test.txt"))
y_test = np.loadtxt(os.path.join(TEST_DIR, "y_test.txt")).astype(int)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (7352, 561) (7352,)
Test shape: (2947, 561) (2947,)


### 🧩 Step 2 — Define period label groups and final class remapping
We will predefine the class mappings for each period to ensure consistency across the experiment.

In [4]:
# Period-to-original-label mapping
period_label_map = {
    1: [4, 5],           # SITTING, STANDING
    2: [4, 5, 1, 2, 3],  # + WALKING variants (merged)
    3: [4, 5, 1, 2, 3, 6],  # + LAYING
    4: [4, 5, 1, 2, 3, 6]   # same classes, but with walking variants separated
}

# Consistent label remapping across all periods (final label index)
final_class_map = {
    "SITTING": 0,
    "STANDING": 1,
    "WALKING": 2,
    "LAYING": 3,
    "WALKING_UPSTAIRS": 4,
    "WALKING_DOWNSTAIRS": 5
}

# Reverse mapping for readability
label_name_from_id = {v: k for k, v in final_class_map.items()}
activity_label_map = {
    1: "WALKING",
    2: "WALKING_UPSTAIRS",
    3: "WALKING_DOWNSTAIRS",
    4: "SITTING",
    5: "STANDING",
    6: "LAYING"
}


### 🧪 Step 3 — Filter and remap datasets for each period
We will build training and testing splits for each period using consistent class indices.

In [5]:
def map_label(label, period):
    name = activity_label_map[label]
    if period < 4:
        if name.startswith("WALKING"):
            return final_class_map["WALKING"]
        else:
            return final_class_map[name]
    else:
        return final_class_map[name]

def get_period_dataset(X, y, period):
    allowed_labels = period_label_map[period]
    mask = np.isin(y, allowed_labels)
    Xp = X[mask]
    yp = np.array([map_label(label, period) for label in y[mask]])
    return Xp, yp


### 📊 Step 4 — Print artistic class distribution for each period
We show each period's label map and class statistics to confirm correctness.

In [6]:
def print_class_distribution(y, var_name: str, label_map: dict) -> None:
    y = np.array(y).flatten()
    unique, counts = np.unique(y, return_counts=True)
    total = counts.sum()
    print(f"\n📦 Class Distribution for {var_name}")
    for i, c in zip(unique, counts):
        print(f"  ├─ Label {i:<2} ({label_map[i]:<20}) → {c:>5} samples ({(c/total)*100:>5.2f}%)")


In [7]:
period_datasets = {}

for period in range(1, 5):
    Xp_train, yp_train = get_period_dataset(X_train, y_train, period)
    Xp_test, yp_test = get_period_dataset(X_test, y_test, period)

    period_datasets[period] = {
        "train": (Xp_train, yp_train),
        "test": (Xp_test, yp_test)
    }

    print_class_distribution(yp_train, f"Period {period} (Train)", label_name_from_id)
    print_class_distribution(yp_test, f"Period {period} (Test)", label_name_from_id)



📦 Class Distribution for Period 1 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (48.35%)
  ├─ Label 1  (STANDING            ) →  1374 samples (51.65%)

📦 Class Distribution for Period 1 (Test)
  ├─ Label 0  (SITTING             ) →   491 samples (48.00%)
  ├─ Label 1  (STANDING            ) →   532 samples (52.00%)

📦 Class Distribution for Period 2 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (21.63%)
  ├─ Label 1  (STANDING            ) →  1374 samples (23.11%)
  ├─ Label 2  (WALKING             ) →  3285 samples (55.26%)

📦 Class Distribution for Period 2 (Test)
  ├─ Label 0  (SITTING             ) →   491 samples (20.37%)
  ├─ Label 1  (STANDING            ) →   532 samples (22.07%)
  ├─ Label 2  (WALKING             ) →  1387 samples (57.55%)

📦 Class Distribution for Period 3 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (17.49%)
  ├─ Label 1  (STANDING            ) →  1374 samples (18.69%)
  ├─ Label 2  (WALKING             ) →  328

## __Check GPU, CUDA, Pytorch__

### GPU Details

In [8]:
!nvidia-smi

Thu Apr 24 23:24:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:2A:00.0 Off |                  Off |
| 30%   44C    P5             77W /  300W |    4539MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [9]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.5.1
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA RTX A6000
Number of GPUs           : 3
Current Device Index     : 0
Compute Capability       : 8.6
Total CUDA Cores         : 10752
Total Memory (GB)        : 47.41
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [10]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.5.1
CUDA Compiled Version    : 12.1
CUDA Available           : Yes
Number of GPUs           : 3
GPU Name                 : NVIDIA RTX A6000
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __⚙️ GPU Selection — Auto-select the least loaded GPU__
This code automatically scans available GPUs and selects the one with the lowest current memory usage.


In [11]:
def auto_select_cuda_device(verbose=True):
    """
    Automatically selects the CUDA GPU with the least memory usage.
    Falls back to CPU if no GPU is available.
    """
    if not torch.cuda.is_available():
        print("🚫 No CUDA GPU available. Using CPU.")
        return torch.device("cpu")

    try:
        # Run nvidia-smi to get memory usage of each GPU
        smi_output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_used = [int(x) for x in smi_output.strip().split('\n')]
        best_gpu = int(np.argmin(memory_used))

        if verbose:
            print("🎯 Automatically selected GPU:")
            print(f"    - CUDA Device ID : {best_gpu}")
            print(f"    - Memory Used    : {memory_used[best_gpu]} MiB")
            print(f"    - Device Name    : {torch.cuda.get_device_name(best_gpu)}")
        return torch.device(f"cuda:{best_gpu}")
    except Exception as e:
        print(f"⚠️ Failed to auto-detect GPU. Falling back to cuda:0. ({e})")
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Execute and assign
device = auto_select_cuda_device()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 4539 MiB
    - Device Name    : NVIDIA RTX A6000


## __MLP Model__

### HAR_MLP_v2

In [12]:
class HAR_MLP_v2(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, dropout: float = 0.2):
        super(HAR_MLP_v2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        return x


## __PNN Module__

In [56]:
# =============================
# PNN_fc1 Implementation for HAR_MLP_v2 
# =============================
class HAR_PNN_Column_fc1(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, new_output_size: int, dropout: float = 0.2):
        super(HAR_PNN_Column_fc1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.fc3 = nn.Linear(hidden_size, new_output_size)
        self.lateral_adapter = nn.Linear(hidden_size, hidden_size)

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor, lateral_features: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        lateral = self.lateral_adapter(lateral_features)
        x = x + lateral

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        out = self.fc3(x)
        return out


class ProgressiveNeuralNetwork_fc1(nn.Module):
    def __init__(self, base_model: nn.Module, new_column: HAR_PNN_Column_fc1):
        super(ProgressiveNeuralNetwork_fc1, self).__init__()
        self.base_model = base_model
        self.new_column = new_column

        for param in self.base_model.parameters():
            param.requires_grad = False

    def forward_features(self, x: torch.Tensor) -> torch.Tensor:
        # ✅ 遞迴支援 base_model 的 forward_features
        with torch.no_grad():
            if hasattr(self.base_model, "forward_features"):
                return self.base_model.forward_features(x)
            else:
                x = self.base_model.fc1(x)
                x = self.base_model.bn1(x)
                x = self.base_model.relu1(x)
                x = self.base_model.dropout1(x)
                return x

    def get_logits(self, x: torch.Tensor, base_features: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            # 遞迴收集所有 base logits
            if isinstance(self.base_model, ProgressiveNeuralNetwork_fc1):
                logits = self.base_model(x)  # ✅ 直接 forward base model（內含其 own logits）
            else:
                x = self.base_model.fc2(base_features)
                x = self.base_model.bn2(x)
                x = self.base_model.relu2(x)
                x = self.base_model.dropout2(x)
                logits = self.base_model.fc3(x)
            return logits

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        base_features = self.forward_features(x)
        base_logits = self.get_logits(x, base_features)
        new_logits = self.new_column(x, lateral_features=base_features)
        return torch.cat([base_logits, new_logits], dim=-1)

In [60]:
# =============================
# PNN_fc2 Implementation for HAR_MLP_v2
# =============================
class HAR_PNN_Column_fc2(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, new_output_size: int, dropout: float = 0.2):
        super(HAR_PNN_Column_fc2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.fc3 = nn.Linear(hidden_size, new_output_size)
        self.lateral_adapter = nn.Linear(hidden_size, hidden_size)

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor, lateral_features: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        lateral = self.lateral_adapter(lateral_features)
        x = x + lateral

        out = self.fc3(x)
        return out


class ProgressiveNeuralNetwork_fc2(nn.Module):
    def __init__(self, base_model: nn.Module, new_column: HAR_PNN_Column_fc2):
        super(ProgressiveNeuralNetwork_fc2, self).__init__()
        self.base_model = base_model
        self.new_column = new_column

        for param in self.base_model.parameters():
            param.requires_grad = False

    def forward_features(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            # 遞迴下探 base_model 若有 forward_features
            if hasattr(self.base_model, "forward_features"):
                return self.base_model.forward_features(x)
            else:
                x = self.base_model.fc1(x)
                x = self.base_model.bn1(x)
                x = self.base_model.relu1(x)
                x = self.base_model.dropout1(x)

                x = self.base_model.fc2(x)
                x = self.base_model.bn2(x)
                x = self.base_model.relu2(x)
                x = self.base_model.dropout2(x)
                return x

    def get_logits(self, x: torch.Tensor, base_features: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            # 遞迴收集所有 base logits
            if isinstance(self.base_model, ProgressiveNeuralNetwork_fc2):
                logits = self.base_model(x)  # ✅ 直接 forward base model（內含其 own logits）
            else:
                logits = self.base_model.fc3(base_features)
            return logits

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        base_features = self.forward_features(x)
        base_logits = self.get_logits(x, base_features)
        new_logits = self.new_column(x, lateral_features=base_features)
        return torch.cat([base_logits, new_logits], dim=-1)

## __Training Function with PNN__

In [16]:
def compute_classwise_accuracy(preds, targets, class_correct, class_total):
    """
    Computes per-class accuracy statistics from raw logits and ground truth.
    """
    preds = torch.argmax(preds, dim=-1)
    correct_mask = (preds == targets)
    for label in torch.unique(targets):
        label = label.item()
        label_mask = (targets == label)
        class_total[label] = class_total.get(label, 0) + label_mask.sum().item()
        class_correct[label] = class_correct.get(label, 0) + (correct_mask & label_mask).sum().item()

In [ ]:
def compute_fwt_har(previous_model, init_model, X_val, y_val, known_classes, batch_size=64):
    """
    FWT computation for HAR-style inputs (tabular MLP, not sequence).
    X_val: shape [N, F]
    y_val: shape [N]
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    previous_model.to(device).eval()
    init_model.to(device).eval()

    # 只選擇 known_classes
    mask = np.isin(y_val, known_classes)
    X_known = X_val[mask]
    y_known = y_val[mask]

    if len(y_known) == 0:
        print(f"⚠️ No validation samples for known classes {known_classes}.")
        return None, None, None

    print(f"📋 Total samples for known classes {known_classes}: {len(y_known)}")

    dataset = TensorDataset(
        torch.tensor(X_known, dtype=torch.float32),
        torch.tensor(y_known, dtype=torch.long)
    )
    loader = DataLoader(dataset, batch_size=batch_size)

    correct_prev, correct_init, total = 0, 0, 0

    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)

            out_prev = previous_model(xb)
            out_init = init_model(xb)

            preds_prev = torch.argmax(out_prev, dim=-1)
            preds_init = torch.argmax(out_init, dim=-1)

            correct_prev += (preds_prev == yb).sum().item()
            correct_init += (preds_init == yb).sum().item()
            total += yb.size(0)

    acc_prev = correct_prev / total
    acc_init = correct_init / total
    fwt_value = acc_prev - acc_init

    print(f"\n### 🔍 FWT Debug Info (HAR):")
    print(f"- Total evaluated samples: {total}")
    print(f"- Correct (PrevModel): {correct_prev} / {total} → Acc = {acc_prev:.4f}")
    print(f"- Correct (InitModel): {correct_init} / {total} → Acc = {acc_init:.4f}")
    print(f"- FWT = Acc_prev - Acc_init = {fwt_value:.4f}")

    return fwt_value, acc_prev, acc_init


In [ ]:
def print_model_info(model):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = total_params * 4  # 假設 float32，每個參數佔 4 bytes
    param_size_MB = param_size_bytes / (1024**2)

    print(f"- Total Parameters: {total_params}")
    print(f"- Model Size (float32): {param_size_MB:.2f} MB")

In [38]:
def train_with_pnn(model, output_size, criterion, optimizer,
                   X_train, y_train, X_val, y_val,
                   scheduler=None, num_epochs=10, batch_size=32,
                   model_saving_folder=None, model_name=None,
                   stop_signal_file=None, period=1, device=auto_select_cuda_device()):
    print("\n🚀 'train_with_pnn' started.")
    model_name = model_name or 'pnn_model'
    model_saving_folder = model_saving_folder or './saved_models'

    if model_saving_folder:
        if os.path.exists(model_saving_folder):
            shutil.rmtree(model_saving_folder)
            print(f"✅ Removed existing folder: {model_saving_folder}")
        os.makedirs(model_saving_folder, exist_ok=True)

    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.long).to(device)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    print("\n✅ Data Overview:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    start_time = time.time()
    best_results = []

    for epoch in range(num_epochs):
        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\n🛑 Stop signal detected. Exiting training loop.")
            break

        model.train()
        epoch_loss = 0.0
        class_correct, class_total = {}, {}

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            logits = model(X_batch).view(-1, output_size)
            y_batch = y_batch.view(-1)
            loss = criterion(logits, y_batch)
            compute_classwise_accuracy(logits, y_batch, class_correct, class_total)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * X_batch.size(0)

        train_loss = epoch_loss / len(train_loader.dataset)
        train_acc = {int(c): f"{(class_correct[c] / class_total[c]) * 100:.2f}%" if class_total[c] > 0 else "0.00%" for c in sorted(class_total.keys())}

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_class_correct, val_class_total = {}, {}

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                logits = model(X_batch).view(-1, output_size)
                y_batch = y_batch.view(-1)
                val_loss += criterion(logits, y_batch).item() * X_batch.size(0)
                predictions = torch.argmax(logits, dim=-1)
                val_correct += (predictions == y_batch).sum().item()
                val_total += y_batch.size(0)
                compute_classwise_accuracy(logits, y_batch, val_class_correct, val_class_total)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_acc_cls = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%" for c in sorted(val_class_total.keys())}

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Train-Class-Acc: {train_acc}")
        print(f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc * 100:.2f}%, Val-Class-Acc: {val_acc_cls}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        model_path = os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        current = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'train_classwise_accuracy': train_acc,
            'val_classwise_accuracy': val_acc_cls,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_path': model_path
        }

        if len(best_results) < 5 or val_acc > best_results[-1]['val_accuracy']:
            if len(best_results) == 5:
                to_remove = best_results.pop()
                if os.path.exists(to_remove['model_path']):
                    os.remove(to_remove['model_path'])
                    print(f"🗑 Removed: {to_remove['model_path']}")
            best_results.append(current)
            best_results.sort(key=lambda x: (x['val_accuracy'], x['epoch']), reverse=True)
            torch.save(current, model_path)
            print(f"✅ Saved model: {model_path}")

        if scheduler:
            scheduler.step(val_loss)

    elapsed_time = time.time() - start_time
    print(f"\n⏳ Total training time: {elapsed_time:.2f} seconds")

    if best_results:
        best = best_results[0]
        best_model_path = os.path.join(model_saving_folder, f"{model_name}_best.pth")
        torch.save(best, best_model_path)
        print(f"\n🏆 Best model saved as: {best_model_path} (Val Accuracy: {best['val_accuracy'] * 100:.2f}%)")

    final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
    torch.save(current, final_model_path)
    print(f"\n📌 Final model saved as: {final_model_path}")

    print("\n🎯 Top 5 Best Models:")
    for res in best_results:
        print(f"Epoch {res['epoch']}, Train Loss: {res['train_loss']:.6f}, Train-Acc: {res['train_classwise_accuracy']},\n"
              f"Val Loss: {res['val_loss']:.6f}, Val Acc: {res['val_accuracy']*100:.2f}%, Val-Acc: {res['val_classwise_accuracy']},"
              f" Model Path: {res['model_path']}")

    model_name_str = model.__class__.__name__
    best_model = max(best_results, key=lambda x: x['val_accuracy'])

    best_md_summary = f"""
    ---
### Period {period}
+ ##### Total training time: {elapsed_time:.2f} seconds
+ ##### Model: {model_name_str}
+ ##### Training and saving in *'{model_saving_folder}'*
+ ##### Best Epoch: {best_model['epoch']}
#### __Val Accuracy: {best_model['val_accuracy'] * 100:.2f}%__
#### __Val-Class-Acc: {best_model['val_classwise_accuracy']}__
    """
    print(best_md_summary.strip())

    del X_train, y_train, X_val, y_val, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 7025 MiB
    - Device Name    : NVIDIA RTX A6000


## __📋 Label Mapping (HAR Activities)__

| Label ID | Activity Name       |
|----------|---------------------|
|    0     | SITTING             |
|    1     | STANDING            |
|    2     | WALKING             |
|    3     | LAYING              |
|    4     | WALKING_UPSTAIRS    |
|    5     | WALKING_DOWNSTAIRS  |

## __Common Parameters__

In [18]:
batch_size = 32
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'HAR_CIL/stop_training.txt'))

## __PNN Baseline Training For PNN_fc1, PNN_fc_2__

---
### Period 1
+ ##### Total training time: 159.83 seconds
+ ##### Model: HAR_MLP_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/Baseline_1'*
+ ##### Best Epoch: 866
#### __Val Accuracy: 95.31%__
#### __Val-Class-Acc: {0: '93.48%', 1: '96.99%'}__

In [23]:
# ================================
# 📌 Period 1 Configuration - HAR_MLP_v2 (PNN Baseline)
# ================================
period = 1

# Load dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Device
device = auto_select_cuda_device()

# Model and training hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000

# ==== Saving Path ====
model_name = "HAR_MLP_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"Baseline_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# ==== Model (Baseline HAR MLP for PNN) ====
model = HAR_MLP_v2(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dropout=dropout
).to(device)

# ==== Optimizer / Criterion / Scheduler ====
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# ==== Training ====
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 25663 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_pnn' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/Baseline_1

✅ Data Overview:
X_train: torch.Size([2660, 561]), y_train: torch.Size([2660])
X_val: torch.Size([1023, 561]), y_val: torch.Size([1023])
Epoch 1/1000, Train Loss: 0.690389, Train-Class-Acc: {0: '46.58%', 1: '79.77%'}
Val Loss: 0.474428, Val Acc: 75.95%, Val-Class-Acc: {0: '56.01%', 1: '94.36%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/Baseline_1/HAR_MLP_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 0.457192, Train-Class-Acc: {0: '72.63%', 1: '85.30%'}
Val Loss: 0.381584, Val Acc: 84.16%, Val-Class-Acc: {0: '94.09%', 1: '75.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/Baseline_1/HAR_MLP_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.375725, Train-Class-Ac

## __~~~ PNN_fc1 ~~~__

---
### Period 2
+ ##### Total training time: 408.46 seconds
+ ##### Model: ProgressiveNeuralNetwork_fc1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_2'*
+ ##### Best Epoch: 636
#### __Val Accuracy: 96.93%__
#### __Val-Class-Acc: {0: '87.17%', 1: '97.93%', 2: '100.00%'}__

In [69]:
# ================================
# 📌 Period 2 Configuration - Progressive Neural Network (PNN_fc1)
# ================================
period = 2

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
new_output_size = 1  # 新增 1 個類別
output_size = len(set(y_train))  # 到目前為止所有類別數

# === Load frozen Period 1 model ===
frozen_model = HAR_MLP_v2(input_size, hidden_size, output_size - new_output_size, dropout)
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"Baseline_{period-1}", "HAR_MLP_PNN_v2_best.pth"
)
frozen_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(frozen_checkpoint['model_state_dict'])
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False

# === Create new PNN column ===
new_column = HAR_PNN_Column_fc1(input_size=input_size, hidden_size=hidden_size, new_output_size=new_output_size)

# === Wrap into ProgressiveNeuralNetwork ===
model = ProgressiveNeuralNetwork_fc1(base_model=frozen_model, new_column=new_column).to(device)

# === Define optimizer, loss, etc. ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC1_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, frozen_model, new_column, frozen_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 4906 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_pnn' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_2

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])


/tmp/ipykernel_2803466/2098380930.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frozen_checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1000, Train Loss: 1.150666, Train-Class-Acc: {0: '90.67%', 1: '98.62%', 2: '46.27%'}
Val Loss: 0.600781, Val Acc: 80.91%, Val-Class-Acc: {0: '79.23%', 1: '99.25%', 2: '74.48%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_2/HAR_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 0.487802, Train-Class-Acc: {0: '92.30%', 1: '98.62%', 2: '75.31%'}
Val Loss: 0.304398, Val Acc: 90.50%, Val-Class-Acc: {0: '86.35%', 1: '98.12%', 2: '89.04%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_2/HAR_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.327222, Train-Class-Acc: {0: '91.29%', 1: '98.33%', 2: '83.81%'}
Val Loss: 0.309093, Val Acc: 90.83%, Val-Class-Acc: {0: '79.43%', 1: '99.25%', 2: '91.64%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_2/HAR_PNN_v2_epoch_3.pth
Epoch 4/1000, Train Loss: 0.222632, Train-Class-Acc: {0: '92.53%', 1: '98

---
### Period 3
+ ##### Total training time: 712.79 seconds
+ ##### Model: ProgressiveNeuralNetwork_fc1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_3'*
+ ##### Best Epoch: 536
#### __Val Accuracy: 97.46%__
#### __Val-Class-Acc: {0: '87.78%', 1: '97.18%', 2: '100.00%', 3: '100.00%'}__

In [70]:
# ================================
# 📌 Period 3 Configuration - Progressive Neural Network (PNN_fc1)
# ================================
period = 3

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
new_output_size = 1  # 新增一個類別
output_size = len(set(y_train))  # 目前總類別數（含舊+新）
print(output_size)
# === 還原 Period 2 架構 ===
base_model = HAR_MLP_v2(input_size, hidden_size, output_size - 2, dropout)
prev_column = HAR_PNN_Column_fc1(input_size, hidden_size, new_output_size=1)

frozen_model = ProgressiveNeuralNetwork_fc1(base_model=base_model, new_column=prev_column)

# === 載入 Period 2 的 model state_dict ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC1_Period_{period-1}", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(checkpoint['model_state_dict'], strict=True)
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False

# === Create Period 3 New Column ===
new_column = HAR_PNN_Column_fc1(input_size, hidden_size, new_output_size=new_output_size)

# === Combine into new PNN wrapper ===
model = ProgressiveNeuralNetwork_fc1(base_model=frozen_model, new_column=new_column).to(device)
print(model)

# === Optimizer / Loss / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC1_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, base_model, prev_column, new_column, frozen_model, checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 4815 MiB
    - Device Name    : NVIDIA RTX A6000
4
ProgressiveNeuralNetwork_fc1(
  (base_model): ProgressiveNeuralNetwork_fc1(
    (base_model): HAR_MLP_v2(
      (fc1): Linear(in_features=561, out_features=128, bias=True)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.2, inplace=False)
      (fc2): Linear(in_features=128, out_features=128, bias=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (dropout2): Dropout(p=0.2, inplace=False)
      (fc3): Linear(in_features=128, out_features=2, bias=True)
    )
    (new_column): HAR_PNN_Column_fc1(
      (fc1): Linear(in_features=561, out_features=128, bias=True)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (dropout1

/tmp/ipykernel_2803466/988963645.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1000, Train Loss: 0.252808, Train-Class-Acc: {0: '88.72%', 1: '98.98%', 2: '100.00%', 3: '72.21%'}
Val Loss: 0.258338, Val Acc: 94.91%, Val-Class-Acc: {0: '82.08%', 1: '99.25%', 2: '100.00%', 3: '89.20%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_3/HAR_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 0.109029, Train-Class-Acc: {0: '89.50%', 1: '99.27%', 2: '100.00%', 3: '93.82%'}
Val Loss: 0.194260, Val Acc: 95.79%, Val-Class-Acc: {0: '82.89%', 1: '99.25%', 2: '100.00%', 3: '93.30%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_3/HAR_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.079023, Train-Class-Acc: {0: '89.58%', 1: '99.42%', 2: '100.00%', 3: '96.94%'}
Val Loss: 0.151489, Val Acc: 96.40%, Val-Class-Acc: {0: '85.95%', 1: '97.74%', 2: '100.00%', 3: '95.34%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_3/HAR_PNN_v2_e

---
### Period 4
+ ##### Total training time: 868.30 seconds
+ ##### Model: ProgressiveNeuralNetwork_fc1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_4'*
+ ##### Best Epoch: 780
#### __Val Accuracy: 96.47%__
#### __Val-Class-Acc: {0: '86.76%', 1: '97.56%', 2: '99.80%', 3: '100.00%', 4: '95.75%', 5: '98.81%'}__

In [71]:
# ================================
# 📌 Period 4 Configuration - Progressive Neural Network (PNN_fc1)
# ================================
period = 4

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
new_output_size = 2  # 新增兩個類別（拆分 WALKING 類）
output_size = len(set(y_train))  # 到目前為止總類別數（應為 5）

# === Step 1: 還原 Period 2 的基礎 MLP + Column 結構 ===
base_model = HAR_MLP_v2(input_size, hidden_size, 2, dropout)  # output: 2
prev_column_2 = HAR_PNN_Column_fc1(input_size, hidden_size, new_output_size=1)  # output: 1
frozen_step1 = ProgressiveNeuralNetwork_fc1(base_model=base_model, new_column=prev_column_2)

# === Step 2: 再加上 Period 3 的 Column ===
prev_column_3 = HAR_PNN_Column_fc1(input_size, hidden_size, new_output_size=1)  # Period 3 新增類別
frozen_step2 = ProgressiveNeuralNetwork_fc1(base_model=frozen_step1, new_column=prev_column_3)

# === 載入 Period 3 模型的 state_dict（已經是 Step2 完整結構）===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC1_Period_{period-1}", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_step2.load_state_dict(checkpoint['model_state_dict'], strict=True)
frozen_step2.to(device)
frozen_step2.eval()
for param in frozen_step2.parameters():
    param.requires_grad = False

# === Step 3: 加上 Period 4 的 Column 並包成新模型 ===
new_column_4 = HAR_PNN_Column_fc1(input_size, hidden_size, new_output_size=new_output_size)
model = ProgressiveNeuralNetwork_fc1(base_model=frozen_step2, new_column=new_column_4).to(device)
print(model)

# === Optimizer / Loss / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC1_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model
del base_model, prev_column_2, prev_column_3, new_column_4, frozen_step1, frozen_step2, checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 4843 MiB
    - Device Name    : NVIDIA RTX A6000
ProgressiveNeuralNetwork_fc1(
  (base_model): ProgressiveNeuralNetwork_fc1(
    (base_model): ProgressiveNeuralNetwork_fc1(
      (base_model): HAR_MLP_v2(
        (fc1): Linear(in_features=561, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (fc2): Linear(in_features=128, out_features=128, bias=True)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (fc3): Linear(in_features=128, out_features=2, bias=True)
      )
      (new_column): HAR_PNN_Column_fc1(
        (fc1): Linear(in_features=561, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, af

/tmp/ipykernel_2803466/913096177.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1000, Train Loss: 5.589476, Train-Class-Acc: {0: '89.27%', 1: '99.27%', 2: '100.00%', 3: '100.00%', 4: '0.00%', 5: '0.00%'}
Val Loss: 4.929336, Val Acc: 67.15%, Val-Class-Acc: {0: '87.17%', 1: '97.37%', 2: '100.00%', 3: '100.00%', 4: '0.00%', 5: '0.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_4/HAR_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 3.519128, Train-Class-Acc: {0: '87.64%', 1: '98.84%', 2: '100.00%', 3: '100.00%', 4: '2.42%', 5: '1.52%'}
Val Loss: 3.225756, Val Acc: 66.00%, Val-Class-Acc: {0: '78.00%', 1: '99.06%', 2: '100.00%', 3: '100.00%', 4: '0.21%', 5: '0.24%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC1_Period_4/HAR_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 2.183600, Train-Class-Acc: {0: '89.35%', 1: '99.27%', 2: '99.92%', 3: '100.00%', 4: '17.99%', 5: '16.02%'}
Val Loss: 1.965656, Val Acc: 73.33%, Val-Class-Acc: {0: '85.74%', 1: '98.12%', 2: '100.00%', 

## __~~~ PNN_fc2 ~~~__

---
### Period 2
+ ##### Total training time: 480.26 seconds
+ ##### Model: ProgressiveNeuralNetwork_fc2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_2'*
+ ##### Best Epoch: 145
#### __Val Accuracy: 96.93%__
#### __Val-Class-Acc: {0: '87.17%', 1: '97.93%', 2: '100.00%'}__

In [72]:
# ================================
# 📌 Period 2 Configuration - Progressive Neural Network (PNN_fc2)
# ================================
period = 2

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
new_output_size = 1  # 新增 1 個類別
output_size = len(set(y_train))  # 到目前為止所有類別數

# === Load frozen Period 1 model ===
frozen_model = HAR_MLP_v2(input_size, hidden_size, output_size - new_output_size, dropout)
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"Baseline_{period-1}", "HAR_MLP_PNN_v2_best.pth"
)
frozen_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(frozen_checkpoint['model_state_dict'])
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False

# === Create new PNN column ===
new_column = HAR_PNN_Column_fc2(input_size=input_size, hidden_size=hidden_size, new_output_size=new_output_size)

# === Wrap into ProgressiveNeuralNetwork ===
model = ProgressiveNeuralNetwork_fc2(base_model=frozen_model, new_column=new_column).to(device)

# === Define optimizer, loss, etc. ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, frozen_model, new_column, frozen_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5303 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_pnn' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_2

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])


/tmp/ipykernel_2803466/3489795576.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frozen_checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1000, Train Loss: 0.380569, Train-Class-Acc: {0: '89.42%', 1: '97.67%', 2: '82.98%'}
Val Loss: 0.221704, Val Acc: 94.94%, Val-Class-Acc: {0: '79.43%', 1: '98.87%', 2: '98.92%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_2/HAR_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 0.094197, Train-Class-Acc: {0: '90.44%', 1: '97.96%', 2: '98.93%'}
Val Loss: 0.158848, Val Acc: 96.47%, Val-Class-Acc: {0: '85.13%', 1: '97.74%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_2/HAR_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.090061, Train-Class-Acc: {0: '90.12%', 1: '98.54%', 2: '99.60%'}
Val Loss: 0.154991, Val Acc: 96.68%, Val-Class-Acc: {0: '85.34%', 1: '98.50%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_2/HAR_PNN_v2_epoch_3.pth
Epoch 4/1000, Train Loss: 0.070391, Train-Class-Acc: {0: '91.45%', 1: '

---
### Period 3
+ ##### Total training time: 729.13 seconds
+ ##### Model: ProgressiveNeuralNetwork_fc2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_3'*
+ ##### Best Epoch: 121
#### __Val Accuracy: 97.56%__
#### __Val-Class-Acc: {0: '87.78%', 1: '97.74%', 2: '100.00%', 3: '100.00%'}__

In [73]:
# ================================
# 📌 Period 3 Configuration - Progressive Neural Network (PNN_fc2)
# ================================
period = 3

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
new_output_size = 1  # 新增一個類別
output_size = len(set(y_train))  # 目前總類別數（含舊+新）

# === 還原 Period 2 架構 ===
base_model = HAR_MLP_v2(input_size, hidden_size, output_size - 2, dropout)
prev_column = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=1)

frozen_model = ProgressiveNeuralNetwork_fc2(base_model=base_model, new_column=prev_column)

# === 載入 Period 2 的 model state_dict ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period-1}", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(checkpoint['model_state_dict'], strict=True)
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False

# === Create Period 3 New Column ===
new_column = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=new_output_size)

# === Combine into new PNN wrapper ===
model = ProgressiveNeuralNetwork_fc2(base_model=frozen_model, new_column=new_column).to(device)
print(model)

# === Optimizer / Loss / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, base_model, prev_column, new_column, frozen_model, checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5305 MiB
    - Device Name    : NVIDIA RTX A6000
ProgressiveNeuralNetwork_fc2(
  (base_model): ProgressiveNeuralNetwork_fc2(
    (base_model): HAR_MLP_v2(
      (fc1): Linear(in_features=561, out_features=128, bias=True)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.2, inplace=False)
      (fc2): Linear(in_features=128, out_features=128, bias=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (dropout2): Dropout(p=0.2, inplace=False)
      (fc3): Linear(in_features=128, out_features=2, bias=True)
    )
    (new_column): HAR_PNN_Column_fc2(
      (fc1): Linear(in_features=561, out_features=128, bias=True)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (dropout1):

/tmp/ipykernel_2803466/4077336448.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1000, Train Loss: 0.210756, Train-Class-Acc: {0: '84.21%', 1: '98.69%', 2: '100.00%', 3: '78.89%'}
Val Loss: 0.310711, Val Acc: 93.82%, Val-Class-Acc: {0: '77.60%', 1: '99.25%', 2: '100.00%', 3: '87.34%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_3/HAR_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 0.094583, Train-Class-Acc: {0: '88.10%', 1: '99.27%', 2: '100.00%', 3: '96.02%'}
Val Loss: 0.200956, Val Acc: 95.62%, Val-Class-Acc: {0: '80.65%', 1: '99.25%', 2: '100.00%', 3: '94.41%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_3/HAR_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.079032, Train-Class-Acc: {0: '89.27%', 1: '99.34%', 2: '100.00%', 3: '98.72%'}
Val Loss: 0.183715, Val Acc: 95.86%, Val-Class-Acc: {0: '81.06%', 1: '99.25%', 2: '100.00%', 3: '95.34%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_3/HAR_PNN_v2_e

---
### Period 4
+ ##### Total training time: 893.29 seconds
+ ##### Model: ProgressiveNeuralNetwork_fc2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_4'*
+ ##### Best Epoch: 555
#### __Val Accuracy: 97.01%__
#### __Val-Class-Acc: {0: '88.59%', 1: '97.37%', 2: '99.80%', 3: '100.00%', 4: '98.51%', 5: '97.62%'}__

In [74]:
# ================================
# 📌 Period 4 Configuration - Progressive Neural Network (PNN_fc2)
# ================================
period = 4

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
new_output_size = 2  # 新增兩個類別（拆分 WALKING 類）
output_size = len(set(y_train))  # 到目前為止總類別數（應為 5）

# === Step 1: 還原 Period 2 的基礎 MLP + Column 結構 ===
base_model = HAR_MLP_v2(input_size, hidden_size, 2, dropout)  # output: 2
prev_column_2 = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=1)  # output: 1
frozen_step1 = ProgressiveNeuralNetwork_fc2(base_model=base_model, new_column=prev_column_2)

# === Step 2: 再加上 Period 3 的 Column ===
prev_column_3 = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=1)  # Period 3 新增類別
frozen_step2 = ProgressiveNeuralNetwork_fc2(base_model=frozen_step1, new_column=prev_column_3)

# === 載入 Period 3 模型的 state_dict（已經是 Step2 完整結構）===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period-1}", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_step2.load_state_dict(checkpoint['model_state_dict'], strict=True)
frozen_step2.to(device)
frozen_step2.eval()
for param in frozen_step2.parameters():
    param.requires_grad = False

# === Step 3: 加上 Period 4 的 Column 並包成新模型 ===
new_column_4 = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=new_output_size)
model = ProgressiveNeuralNetwork_fc2(base_model=frozen_step2, new_column=new_column_4).to(device)
print(model)

# === Optimizer / Loss / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model
del base_model, prev_column_2, prev_column_3, new_column_4, frozen_step1, frozen_step2, checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5317 MiB
    - Device Name    : NVIDIA RTX A6000
ProgressiveNeuralNetwork_fc2(
  (base_model): ProgressiveNeuralNetwork_fc2(
    (base_model): ProgressiveNeuralNetwork_fc2(
      (base_model): HAR_MLP_v2(
        (fc1): Linear(in_features=561, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (fc2): Linear(in_features=128, out_features=128, bias=True)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (fc3): Linear(in_features=128, out_features=2, bias=True)
      )
      (new_column): HAR_PNN_Column_fc2(
        (fc1): Linear(in_features=561, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, af

/tmp/ipykernel_2803466/2956673141.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1000, Train Loss: 4.910972, Train-Class-Acc: {0: '81.96%', 1: '92.65%', 2: '99.92%', 3: '100.00%', 4: '1.21%', 5: '2.03%'}
Val Loss: 2.820312, Val Acc: 65.05%, Val-Class-Acc: {0: '76.78%', 1: '94.92%', 2: '100.00%', 3: '100.00%', 4: '0.00%', 5: '0.48%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_4/HAR_PNN_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 1.320140, Train-Class-Acc: {0: '74.42%', 1: '86.83%', 2: '92.41%', 3: '99.86%', 4: '36.25%', 5: '43.20%'}
Val Loss: 0.615264, Val Acc: 83.00%, Val-Class-Acc: {0: '79.84%', 1: '97.37%', 2: '98.79%', 3: '99.63%', 4: '53.29%', 5: '61.90%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/PNN_CIL_v2/PNN_FC2_Period_4/HAR_PNN_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.610914, Train-Class-Acc: {0: '85.07%', 1: '92.87%', 2: '85.48%', 3: '99.79%', 4: '68.22%', 5: '70.08%'}
Val Loss: 0.426050, Val Acc: 88.60%, Val-Class-Acc: {0: '76.17%', 1: '99.06%', 2: '97.58%', 3: 

##  __Compute FWT & Model Size__

### Period 1

In [ ]:
# ================================
# 📌 Period 1 Configuration - HAR_MLP_v2 (PNN Baseline)
# ================================
period = 1

# Load dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Device
device = auto_select_cuda_device()

# Model and training hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1

# ==== Saving Path ====
model_name = "HAR_MLP_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "TT",
    "PNN_CIL_v2", f"Baseline_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# ==== Model (Baseline HAR MLP for PNN) ====
model = HAR_MLP_v2(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dropout=dropout
).to(device)

# ==== Optimizer / Criterion / Scheduler ====
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

print_model_info(model)

# ==== Training ====
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 391 MiB
    - Device Name    : NVIDIA RTX A6000
- Total Parameters: 89218
- Model Size (float32): 0.34 MB

🚀 'train_with_pnn' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/Baseline_1

✅ Data Overview:
X_train: torch.Size([2660, 561]), y_train: torch.Size([2660])
X_val: torch.Size([1023, 561]), y_val: torch.Size([1023])
Epoch 1/1, Train Loss: 0.738479, Train-Class-Acc: {0: '63.69%', 1: '53.35%'}
Val Loss: 0.533027, Val Acc: 72.92%, Val-Class-Acc: {0: '46.44%', 1: '97.37%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/Baseline_1/HAR_MLP_PNN_v2_epoch_1.pth

⏳ Total training time: 0.47 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/Baseline_1/HAR_MLP_PNN_v2_best.pth (Val Accuracy: 72.92%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/Baseline_1/HAR_MLP_PNN_v2_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train

### Period 2

In [ ]:
# ================================
# 📌 FWT - Period 2 (HAR PNN LoRA)
# ================================
period = 2

X_val = period_datasets[period]["test"][0]
y_val = period_datasets[period]["test"][1]

input_size = X_val.shape[1]
hidden_size = 128
output_size_prev = 2  # Period 1 類別：SITTING (0), STANDING (1)
dropout = 0.2
device = auto_select_cuda_device()

# === 載入 Period 1 模型作為 frozen base ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", "Baseline_1", "HAR_MLP_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)

previous_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
previous_model.load_state_dict(checkpoint["model_state_dict"])
previous_model.output_size = output_size_prev
del checkpoint
gc.collect()

# === Init Model（相同結構，但未訓練）===
init_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
init_model.output_size = output_size_prev

# === 準備資料 ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
known_classes = [0, 1]  # Period 1 穩定類別：SITTING, STANDING

# === ✅ 計算 FWT ===
fwt, acc_prev, acc_init = compute_fwt_har(previous_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 2:")
print(f"- FWT (PNN HAR Period 2, old classes {known_classes}): {fwt * 100:.2f}%")
print(f"- Accuracy by previous model: {acc_prev * 100:.2f}%")
print(f"- Accuracy by init model:     {acc_init * 100:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5118 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_1853981/1064505577.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


📋 Total samples for known classes [0, 1]: 1023

### 🔍 FWT Debug Info (HAR):
- Total evaluated samples: 1023
- Correct (PrevModel): 951 / 1023 → Acc = 0.9296
- Correct (InitModel): 465 / 1023 → Acc = 0.4545
- FWT = Acc_prev - Acc_init = 0.4751

### Period 2:
- FWT (PNN HAR Period 2, old classes [0, 1]): 47.51%
- Accuracy by previous model: 92.96%
- Accuracy by init model:     45.45%


/tmp/ipykernel_1853981/3765843572.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_known, dtype=torch.float32),
/tmp/ipykernel_1853981/3765843572.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_known, dtype=torch.long)


In [ ]:
# ================================
# 📌 Period 2 Configuration - Progressive Neural Network (PNN_fc2)
# ================================
period = 2

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
new_output_size = 1  # 新增 1 個類別
output_size = len(set(y_train))  # 到目前為止所有類別數

# === Load frozen Period 1 model ===
frozen_model = HAR_MLP_v2(input_size, hidden_size, output_size - new_output_size, dropout)
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"Baseline_{period-1}", "HAR_MLP_PNN_v2_best.pth"
)
frozen_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(frozen_checkpoint['model_state_dict'])
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False

# === Create new PNN column ===
new_column = HAR_PNN_Column_fc2(input_size=input_size, hidden_size=hidden_size, new_output_size=new_output_size)

# === Wrap into ProgressiveNeuralNetwork ===
model = ProgressiveNeuralNetwork_fc2(base_model=frozen_model, new_column=new_column).to(device)

# === Define optimizer, loss, etc. ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "TT",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)


print_model_info(model)

# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, frozen_model, new_column, frozen_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 651 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_781994/3163962344.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frozen_checkpoint = torch.load(prev_model_path, map_location=device)


- Total Parameters: 194819
- Model Size (float32): 0.74 MB

🚀 'train_with_pnn' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_2

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])
Epoch 1/1, Train Loss: 0.497424, Train-Class-Acc: {0: '89.74%', 1: '97.67%', 2: '78.48%'}
Val Loss: 0.191339, Val Acc: 95.81%, Val-Class-Acc: {0: '81.67%', 1: '98.31%', 2: '99.86%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_2/HAR_PNN_v2_epoch_1.pth

⏳ Total training time: 0.84 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_2/HAR_PNN_v2_best.pth (Val Accuracy: 95.81%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_2/HAR_PNN_v2_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train Loss: 0.497424, Train-Acc: {0: '89.74%', 1: '97.67%', 2: '78.48%'},
Val Loss: 0.191339

### Period 3

In [ ]:
# ================================
# 📌 FWT - Period 3 (HAR PNN)
# ================================
period = 3

X_val = period_datasets[period]["test"][0]
y_val = period_datasets[period]["test"][1]

input_size = X_val.shape[1]
hidden_size = 128
dropout = 0.2
device = auto_select_cuda_device()
output_size_prev = 3  # Period 2: SITTING, STANDING, WALKING → label 0,1,2

# === 還原 Period 2 架構並載入模型 ===
base_model = HAR_MLP_v2(input_size, hidden_size, 2, dropout)
prev_column = HAR_PNN_Column_fc2(input_size, hidden_size, 1)
prev_model = ProgressiveNeuralNetwork_fc2(base_model, prev_column).to(device)

prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", "PNN_FC2_Period_2", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
prev_model.load_state_dict(checkpoint["model_state_dict"])
prev_model.output_size = output_size_prev
del checkpoint
gc.collect()

# === Init model ===
init_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
init_model.output_size = output_size_prev

# === 資料與 FWT ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
known_classes = [0, 1, 2]  # SITTING, STANDING, WALKING

fwt, acc_prev, acc_init = compute_fwt_har(prev_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 3:")
print(f"- FWT (PNN HAR Period 3, old classes {known_classes}): {fwt * 100:.2f}%")
print(f"- Accuracy by previous model: {acc_prev * 100:.2f}%")
print(f"- Accuracy by init model:     {acc_init * 100:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5120 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_1853981/3353869866.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)
/tmp/ipykernel_1853981/3

📋 Total samples for known classes [0, 1, 2]: 2410

### 🔍 FWT Debug Info (HAR):
- Total evaluated samples: 2410
- Correct (PrevModel): 2318 / 2410 → Acc = 0.9618
- Correct (InitModel): 582 / 2410 → Acc = 0.2415
- FWT = Acc_prev - Acc_init = 0.7203

### Period 3:
- FWT (PNN HAR Period 3, old classes [0, 1, 2]): 72.03%
- Accuracy by previous model: 96.18%
- Accuracy by init model:     24.15%


In [ ]:
# ================================
# 📌 Period 3 Configuration - Progressive Neural Network (PNN_fc2)
# ================================
period = 3

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
new_output_size = 1  # 新增一個類別
output_size = len(set(y_train))  # 目前總類別數（含舊+新）

# === 還原 Period 2 架構 ===
base_model = HAR_MLP_v2(input_size, hidden_size, output_size - 2, dropout)
prev_column = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=1)

frozen_model = ProgressiveNeuralNetwork_fc2(base_model=base_model, new_column=prev_column)

# === 載入 Period 2 的 model state_dict ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period-1}", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(checkpoint['model_state_dict'], strict=True)
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False

# === Create Period 3 New Column ===
new_column = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=new_output_size)

# === Combine into new PNN wrapper ===
model = ProgressiveNeuralNetwork_fc2(base_model=frozen_model, new_column=new_column).to(device)
print(model)

# === Optimizer / Loss / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "TT",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

print_model_info(model)
# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, base_model, prev_column, new_column, frozen_model, checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 763 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_781994/4016017670.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


ProgressiveNeuralNetwork_fc2(
  (base_model): ProgressiveNeuralNetwork_fc2(
    (base_model): HAR_MLP_v2(
      (fc1): Linear(in_features=561, out_features=128, bias=True)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.2, inplace=False)
      (fc2): Linear(in_features=128, out_features=128, bias=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (dropout2): Dropout(p=0.2, inplace=False)
      (fc3): Linear(in_features=128, out_features=2, bias=True)
    )
    (new_column): HAR_PNN_Column_fc2(
      (fc1): Linear(in_features=561, out_features=128, bias=True)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.2, inplace=False)
      (fc2): Linear(in_features=128, out_features=128, bias=True)
      (bn2): BatchNorm1d(128, 

### Period 4

In [ ]:
# ================================
# 📌 FWT - Period 4 (HAR PNN)
# ================================
period = 4

X_val = period_datasets[period]["test"][0]
y_val = period_datasets[period]["test"][1]

input_size = X_val.shape[1]
hidden_size = 128
dropout = 0.2
device = auto_select_cuda_device()
output_size_prev = 4  # Period 3: {0,1,2,3} = SITTING, STANDING, WALKING, LAYING

# === 重建 Period 3 架構並載入 ===
base_model = HAR_MLP_v2(input_size, hidden_size, 2, dropout)
prev_column_2 = HAR_PNN_Column_fc2(input_size, hidden_size, 1)
frozen_step1 = ProgressiveNeuralNetwork_fc2(base_model, prev_column_2)

prev_column_3 = HAR_PNN_Column_fc2(input_size, hidden_size, 1)
prev_model = ProgressiveNeuralNetwork_fc2(frozen_step1, prev_column_3).to(device)

prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", "PNN_FC2_Period_3", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
prev_model.load_state_dict(checkpoint["model_state_dict"])
prev_model.output_size = output_size_prev
del checkpoint
gc.collect()

# === Init model ===
init_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
init_model.output_size = output_size_prev

# === 資料與 FWT ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
known_classes = [0, 1, 3]  # Period 4 穩定類別（排除 WALKING 類拆分後不穩定語意）

fwt, acc_prev, acc_init = compute_fwt_har(prev_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 4:")
print(f"- FWT (PNN HAR Period 4, old classes {known_classes}): {fwt * 100:.2f}%")
print(f"- Accuracy by previous model: {acc_prev * 100:.2f}%")
print(f"- Accuracy by init model:     {acc_init * 100:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5120 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_1853981/2821685562.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


📋 Total samples for known classes [0, 1, 3]: 1560

### 🔍 FWT Debug Info (HAR):
- Total evaluated samples: 1560
- Correct (PrevModel): 1458 / 1560 → Acc = 0.9346
- Correct (InitModel): 489 / 1560 → Acc = 0.3135
- FWT = Acc_prev - Acc_init = 0.6212

### Period 4:
- FWT (PNN HAR Period 4, old classes [0, 1, 3]): 62.12%
- Accuracy by previous model: 93.46%
- Accuracy by init model:     31.35%


/tmp/ipykernel_1853981/3765843572.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_known, dtype=torch.float32),
/tmp/ipykernel_1853981/3765843572.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_known, dtype=torch.long)


In [ ]:
# ================================
# 📌 Period 4 Configuration - Progressive Neural Network (PNN_fc2)
# ================================
period = 4

# === Load dataset for this period ===
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# === Device ===
device = auto_select_cuda_device()

# === Training Hyperparameters ===
input_size    = X_train.shape[1]
hidden_size   = 128
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
new_output_size = 2  # 新增兩個類別（拆分 WALKING 類）
output_size = len(set(y_train))  # 到目前為止總類別數（應為 5）

# === Step 1: 還原 Period 2 的基礎 MLP + Column 結構 ===
base_model = HAR_MLP_v2(input_size, hidden_size, 2, dropout)  # output: 2
prev_column_2 = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=1)  # output: 1
frozen_step1 = ProgressiveNeuralNetwork_fc2(base_model=base_model, new_column=prev_column_2)

# === Step 2: 再加上 Period 3 的 Column ===
prev_column_3 = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=1)  # Period 3 新增類別
frozen_step2 = ProgressiveNeuralNetwork_fc2(base_model=frozen_step1, new_column=prev_column_3)

# === 載入 Period 3 模型的 state_dict（已經是 Step2 完整結構）===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period-1}", "HAR_PNN_v2_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_step2.load_state_dict(checkpoint['model_state_dict'], strict=True)
frozen_step2.to(device)
frozen_step2.eval()
for param in frozen_step2.parameters():
    param.requires_grad = False

# === Step 3: 加上 Period 4 的 Column 並包成新模型 ===
new_column_4 = HAR_PNN_Column_fc2(input_size, hidden_size, new_output_size=new_output_size)
model = ProgressiveNeuralNetwork_fc2(base_model=frozen_step2, new_column=new_column_4).to(device)
print(model)

# === Optimizer / Loss / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Saving Path ===
model_name = "HAR_PNN_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "TT",
    "PNN_CIL_v2", f"PNN_FC2_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

print_model_info(model)
# === Training ===
train_with_pnn(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period,
    device=device
)

# === Summary & Cleanup ===
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model
del base_model, prev_column_2, prev_column_3, new_column_4, frozen_step1, frozen_step2, checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 765 MiB
    - Device Name    : NVIDIA RTX A6000
ProgressiveNeuralNetwork_fc2(
  (base_model): ProgressiveNeuralNetwork_fc2(
    (base_model): ProgressiveNeuralNetwork_fc2(
      (base_model): HAR_MLP_v2(
        (fc1): Linear(in_features=561, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (fc2): Linear(in_features=128, out_features=128, bias=True)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (fc3): Linear(in_features=128, out_features=2, bias=True)
      )
      (new_column): HAR_PNN_Column_fc2(
        (fc1): Linear(in_features=561, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, aff

/tmp/ipykernel_781994/3291222569.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


Epoch 1/1, Train Loss: 4.623844, Train-Class-Acc: {0: '79.24%', 1: '91.99%', 2: '99.18%', 3: '100.00%', 4: '0.75%', 5: '5.58%'}
Val Loss: 2.147958, Val Acc: 67.66%, Val-Class-Acc: {0: '79.23%', 1: '93.80%', 2: '100.00%', 3: '100.00%', 4: '0.00%', 5: '17.38%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_4/HAR_PNN_v2_epoch_1.pth

⏳ Total training time: 1.38 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_4/HAR_PNN_v2_best.pth (Val Accuracy: 67.66%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_4/HAR_PNN_v2_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train Loss: 4.623844, Train-Acc: {0: '79.24%', 1: '91.99%', 2: '99.18%', 3: '100.00%', 4: '0.75%', 5: '5.58%'},
Val Loss: 2.147958, Val Acc: 67.66%, Val-Acc: {0: '79.23%', 1: '93.80%', 2: '100.00%', 3: '100.00%', 4: '0.00%', 5: '17.38%'}, Model Path: Class_Incremental_CL/HAR_CIL/TT/PNN_CIL_v2/PNN_FC2_Period_4/HAR_PNN_v2_epoch

---
## 📊 Comprehensive Test Results: 
### HAR - PNN_FC1 (ProgressiveNeuralNetwork_fc1)

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                     |
|--------|-------------------|---------------------|--------------------------------------------------------------------------|
| 1      | 159.83            | **95.31%**          | {0: 93.48%, 1: 96.99%}                                                  |
| 2      | 408.46            | **96.93%**          | {0: 87.17%, 1: 97.93%, 2: 100.00%}                                      |
| 3      | 712.79            | **97.46%**          | {0: 87.78%, 1: 97.18%, 2: 100.00%, 3: 100.00%}                          |
| 4      | 868.30            | **96.47%**          | {0: 86.76%, 1: 97.56%, 2: 99.80%, 3: 100.00%, 4: 95.75%, 5: 98.81%}     |

### ✔️ HAR - PNN_FC2 (ProgressiveNeuralNetwork_fc2)

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                     |
|--------|-------------------|---------------------|--------------------------------------------------------------------------|
| 1      | 159.83            | **95.31%**          | {0: 93.48%, 1: 96.99%}                                                  |
| 2      | 480.26            | **96.93%**          | {0: 87.17%, 1: 97.93%, 2: 100.00%}                                      |
| 3      | 729.13            | **97.56%**          | {0: 87.78%, 1: 97.74%, 2: 100.00%, 3: 100.00%}                          |
| 4      | 893.29            | **97.01%**          | {0: 88.59%, 1: 97.37%, 2: 99.80%, 3: 100.00%, 4: 98.51%, 5: 97.62%}     |

---



## 📊 Summary: 

### ✔️ HAR - PNN: Validation Summary

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                     |
|--------|-------------------|---------------------|--------------------------------------------------------------------------|
| 1      | 149.07            | **95.31%**          | {0: 95.32%, 1: 95.30%}                                                 |
| 2      | 480.26            | **96.93%**          | {0: 87.17%, 1: 97.93%, 2: 100.00%}                                      |
| 3      | 729.13            | **97.56%**          | {0: 87.78%, 1: 97.74%, 2: 100.00%, 3: 100.00%}                          |
| 4      | 893.29            | **97.01%**          | {0: 88.59%, 1: 97.37%, 2: 99.80%, 3: 100.00%, 4: 98.51%, 5: 97.62%}     |


### 🧠 Continual Learning Metrics

| Period | AA_old (%) | AA_new (%) | BWT (%) | FWT (%) | FWT Classes     | Prev. Model Acc | Init Model Acc |
|--------|------------|------------|---------|---------|------------------|------------------|-----------------|
| 2      | 92.55%     | 100.00%    | -2.68%  | 47.51%  | [0, 1]           | 92.96%           | 45.45%          |
| 3      | 95.17%     | 100.00%    | +0.14%  | 72.03%  | [0, 1, 2]        | 96.18%           | 24.15%          |
| 4      | 96.44%     | 98.07%     | +0.06%  | 62.12%  | [0, 1, 3]        | 93.46%           | 31.35%          |


### 📦 Model Size per Period

| Period | Output Size | Total Params | Δ Params vs Prev | Δ % vs Prev | Model Size (float32) |
|--------|-------------|--------------|------------------|-------------|-----------------------|
| 1      | 2           | 89,218       | —                | —           | 0.34 MB               |
| 2      | 3           | 194,819      | +105,601         | +118.34%    | 0.74 MB               |
| 3      | 4           | 300,420      | +105,601         | +54.21%     | 1.15 MB               |
| 4      | 6           | 406,150      | +105,730         | +35.20%     | 1.55 MB               |

**📈 Model Growth Rate (MGR) = (406,150 - 89,218) / (89,218 × 3) ≈ +118.38%**

**📈 Max trainable ratio ≈ 54.20%**